In [1]:
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,classification_report, make_scorer
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from pathlib import Path
import os.path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
image_dir = Path('Fish_Dataset/Fish_Dataset')
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths,labels], axis=1)
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x[-2:] == 'GT' else x)
image_df = image_df.dropna(axis=0)

samples = []
for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    samples.append(category_slice.sample(150, random_state=1))

image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
image_df

,Filepath,Label
0,Fish_Dataset\Fish_Dataset\Red Sea Bream\Red Se...,Red Sea Bream
1,Fish_Dataset\Fish_Dataset\Red Sea Bream\Red Se...,Red Sea Bream
2,Fish_Dataset\Fish_Dataset\Red Mullet\Red Mulle...,Red Mullet
3,Fish_Dataset\Fish_Dataset\Red Mullet\Red Mulle...,Red Mullet
4,Fish_Dataset\Fish_Dataset\Sea Bass\Sea Bass\00...,Sea Bass
...,...,...
1345,Fish_Dataset\Fish_Dataset\Red Sea Bream\Red Se...,Red Sea Bream
1346,Fish_Dataset\Fish_Dataset\Shrimp\Shrimp\00930.png,Shrimp
1347,Fish_Dataset\Fish_Dataset\Striped Red Mullet\S...,Striped Red Mullet
1348,Fish_Dataset\Fish_Dataset\Gilt-Head Bream\Gilt...,Gilt-Head Bream


In [3]:
image_df.to_csv('fishesDT.csv', index=False)

In [4]:
a = np.zeros(shape=(1350,445,590,3))

In [5]:
count = 0
for i in image_df['Filepath']:
    img_path = i
    img_arr = cv2.imread(img_path)
    a[count] = img_arr
    count += 1

a.shape

(1350, 445, 590, 3)

In [6]:
y_train = image_df['Label'].values

In [7]:
#sklearn expects i/p to be 2d array-model.=>reshape to 2d array
nsamples, nx, ny, nrgb = a.shape
X_train = a.reshape((nsamples,nx*ny*nrgb))

In [8]:
def cross_validation(model, _X, _y, _cv=5):
    _scorers = {
        'accuracy_score': make_scorer(accuracy_score),
        'precision_score': make_scorer(precision_score, average='macro'),
        'recall_score': make_scorer(recall_score, average='macro')
    }
    results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scorers,
                               return_train_score=True)

    return {"Training Accuracy scores": results['train_accuracy_score'],
              "Mean Training Accuracy": results['train_accuracy_score'].mean()*100,
              "Training Precision scores": results['train_precision_score'],
              "Mean Training Precision": results['train_precision_score'].mean(),
              "Training Recall scores": results['train_recall_score'],
              "Mean Training Recall": results['train_recall_score'].mean(),
              "Validation Accuracy scores": results['test_accuracy_score'],
              "Mean Validation Accuracy": results['test_accuracy_score'].mean()*100,
              "Validation Precision scores": results['test_precision_score'],
              "Mean Validation Precision": results['test_precision_score'].mean(),
              "Validation Recall scores": results['test_recall_score'],
              "Mean Validation Recall": results['test_recall_score'].mean(),
              }

In [9]:
decision_tree_model = DecisionTreeClassifier(criterion="gini", random_state=0)
decision_tree_result = cross_validation(decision_tree_model, X_train, y_train, 5)

In [ ]:
print(decision_tree_result)

{'Training Accuracy scores': array([1., 1., 1., 1., 1.]), 'Mean Training Accuracy': 100.0, 'Training Precision scores': array([1., 1., 1., 1., 1.]), 'Mean Training Precision': 1.0, 'Training Recall scores': array([1., 1., 1., 1., 1.]), 'Mean Training Recall': 1.0, 'Validation Accuracy scores': array([0.40740741, 0.45185185, 0.38518519, 0.4037037 , 0.43703704]), 'Mean Validation Accuracy': 41.703703703703695, 'Validation Precision scores': array([0.40501302, 0.45817151, 0.39074708, 0.41652473, 0.43753029]), 'Mean Validation Precision': 0.4215973238916978, 'Validation Recall scores': array([0.40740741, 0.45185185, 0.38518519, 0.4037037 , 0.43703704]), 'Mean Validation Recall': 0.417037037037037}
